# BACH Dataset Exploration

[Check this out](https://iciar2018-challenge.grand-challenge.org/Data) for information about the structure of the dataset

In [2]:
import os
import sys

parent = os.path.dirname(os.path.dirname(os.getcwd()))
if parent not in sys.path:
    sys.path.append(parent)
    
bach_train_folder = os.path.join(parent,"data","processed","BACH_TRAIN")

# Structure
There is a photos folder and a WSI (whole slide image) folder. In the photos folder, there are four smaller folders each containing 100 images. They are:
- **Benign** - Non Cancerous
- **InSitu** - Cancerous
- **Invasive** - Cancerous
- **Normal** - Non Cancerous

In [10]:
from PIL import Image
from torchvision.transforms import ToTensor
img = Image.open(os.path.join(bach_train_folder,"Benign","b002.tif"))
#img.show()
img = ToTensor()(img)

# Segmentation Quality

In [ ]:
# MODEL
from src.model.architectures.graph_construction.hover_net import HoVerNet
chkpt = "HoVerNet_MD-1.1.ckpt"
chkpt_path = os.path.join(parent,"experiments","checkpoints",chkpt)

model = HoVerNet.load_from_checkpoint(chkpt_path, RESNET_SIZE=50,START_LR=0)

In [9]:
from src.transforms.graph_construction.hovernet_post_processing import instance_mask_prediction_hovernet,cut_img_from_tile
from src.transforms.graph_construction.percolation import hollow
tile_size=32

ins_pred = instance_mask_prediction_hovernet(model, img,tile_size)
img_trans = cut_img_from_tile(img,tile_size)
hl_pred = hollow(ins_pred)

In [ ]:
from numpy.ma import masked_where
import matplotlib.pyplot as plt
from src.utilities.img_utilities import tensor_to_numpy

f,ax = plt.subplots(1,1,figsize=(20,20))


ax.imshow(tensor_to_numpy(img_trans))
ax.imshow(masked_where(hl_pred!=0,hl_pred),cmap="nipy_spectral",alpha=0.7)
plt.show()